<a href="https://colab.research.google.com/github/Yessense/yolov5n_detect_drone/blob/master/yolo%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DLS project by Alexandr Korchemnyi

In [ ]:
# !git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd yolov5
# %pip install -qr requirements.txt

/home/yessense/projects/dls project/yolov5


In [ ]:
import torch
import os
from IPython.display import Image, clear_output  # to display images

In [ ]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.8.1+cu102 (NVIDIA GeForce RTX 2060)


In [ ]:
dataset_directory = "/home/yessense/projects/dls project/dataset"

In [ ]:
train_len = len(os.listdir(os.path.join(dataset_directory, "train/images")))
test_len = len(os.listdir(os.path.join(dataset_directory, "test/images")))
valid_len = len(os.listdir(os.path.join(dataset_directory, "valid/images")))

print(f'Lenght of train dataset: {train_len}')
print(f'Lenght of validation dataset: {valid_len}')
print(f'Lenght of test dataset: {test_len}')
print(f'Total lenght with reflect augmentation: {sum([train_len, valid_len, test_len])}')


# Данные
Набор данных снят на полигоне для дронов в МИРЭА. Размечен с помощью roboflow.

![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/imgs/train_batch0.jpg?raw=1)

# Тренировка модели
Изначальная модель - yolov5n
Взята, как самая маленькая, чтобы запускать на

In [ ]:
!python train.py --img 320 --batch 16 --epochs 150 --data "/home/yessense/projects/dls project/dataset/data.yaml" --weights yolov5n.pt --cache --hyp "/home/yessense/projects/dls project/dataset/hyp.yaml"

![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/imgs/F1_curve.png?raw=1)
![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/imgs/P_curve.png?raw=1)
![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/imgs/PR_curve.png?raw=1)
![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/imgs/R_curve.png?raw=1)

In [ ]:
!python export.py --img 320 --batch 1 --weights "/home/yessense/projects/dls project/yolov5/runs/train/exp5/weights/best.pt" --include onnx

# Результаты
![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/results/1.jpeg?raw=1)
![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/results/2.jpeg?raw=1)
![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/results/3.jpeg?raw=1)
![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/results/4.jpeg?raw=1)
![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/results/5.jpeg?raw=1)
![](https://github.com/Yessense/yolov5n_detect_drone/blob/master/results/find.gif?raw=1)

